In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_839/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/12 03:49:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/12 03:49:30 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
data = [
    [1, 2, '2016-03-01', 5],
    [1, 2, '2016-03-02', 6],
    [1, 2, '2016-03-03', 6],
    [2, 3, '2017-06-25', 1],
    [3, 1, '2016-03-02', 0],
    [3, 4, '2018-07-03', 5],
]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype(
    {'player_id': 'Int64', 'device_id': 'Int64', 'event_date': 'datetime64[ns]', 'games_played': 'Int64'})


df_person = spark.createDataFrame(activity)
df_person.createOrReplaceTempView("Activity")

In [4]:
activity

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-03-02,6
2,1,2,2016-03-03,6
3,2,3,2017-06-25,1
4,3,1,2016-03-02,0
5,3,4,2018-07-03,5


In [5]:
query_1 = """
    select
        round(count(distinct player_id) / (select count(distinct player_id) from Activity), 2) as fraction
    from (select
        a.player_id,
        a.device_id,
        a.event_date,
        a.games_played,
        lag(a.event_date, 1, a.event_date) over (partition by player_id order by event_date asc) as previous_login
    from Activity a) t
    where 
        date_sub(event_date, 1) = previous_login 
"""

In [6]:
spark.sql(query_1).show()

+--------+
|fraction|
+--------+
|    0.33|
+--------+



In [7]:
data_str = """
| player_id | device_id | event_date | games_played |
| --------- | --------- | ---------- | ------------ |
| 5         | 17        | 2019-01-17 | 44           |
| 31        | 43        | 2019-01-21 | 41           |
| 52        | 89        | 2019-03-09 | 44           |
| 14        | 154       | 2019-02-13 | 87           |
| 96        | 46        | 2019-03-22 | 77           |
| 73        | 131       | 2019-03-19 | 70           |
| 49        | 106       | 2019-02-13 | 80           |
| 12        | 189       | 2019-01-15 | 13           |
| 40        | 151       | 2019-01-19 | 4            |
| 63        | 196       | 2019-02-23 | 74           |
| 37        | 86        | 2019-01-22 | 38           |
| 60        | 41        | 2019-03-07 | 46           |
| 60        | 162       | 2019-01-12 | 8            |
| 9         | 81        | 2019-01-28 | 88           |
| 83        | 48        | 2019-02-15 | 6            |
| 97        | 85        | 2019-03-30 | 0            |
| 21        | 87        | 2019-01-18 | 16           |
| 77        | 170       | 2019-02-17 | 41           |
| 37        | 192       | 2019-03-04 | 37           |
| 76        | 141       | 2019-01-04 | 3            |
| 89        | 48        | 2019-02-26 | 80           |
| 48        | 3         | 2019-03-25 | 70           |
| 68        | 165       | 2019-02-07 | 12           |
| 63        | 200       | 2019-03-11 | 73           |
| 84        | 126       | 2019-02-04 | 73           |
| 57        | 23        | 2019-02-16 | 16           |
| 36        | 19        | 2019-02-27 | 54           |
| 73        | 7         | 2019-03-28 | 54           |
| 17        | 172       | 2019-03-14 | 67           |
| 46        | 143       | 2019-01-28 | 43           |
| 56        | 79        | 2019-01-08 | 99           |
| 31        | 44        | 2019-03-09 | 78           |
| 58        | 48        | 2019-03-08 | 46           |
| 87        | 111       | 2019-02-20 | 17           |
| 3         | 10        | 2019-02-20 | 31           |
| 44        | 19        | 2019-01-12 | 37           |
| 95        | 89        | 2019-03-07 | 4            |
| 7         | 151       | 2019-03-04 | 66           |
| 66        | 16        | 2019-03-01 | 73           |
| 37        | 132       | 2019-03-08 | 52           |
| 26        | 116       | 2019-02-24 | 34           |
| 7         | 126       | 2019-03-09 | 64           |
| 98        | 170       | 2019-03-05 | 23           |
| 91        | 50        | 2019-01-26 | 32           |
| 47        | 160       | 2019-02-12 | 99           |
| 52        | 83        | 2019-03-23 | 35           |
| 45        | 106       | 2019-01-28 | 39           |
| 34        | 94        | 2019-02-24 | 60           |
| 75        | 110       | 2019-03-13 | 66           |
| 77        | 131       | 2019-01-11 | 12           |
| 12        | 68        | 2019-03-22 | 68           |
| 40        | 97        | 2019-02-11 | 62           |
| 91        | 86        | 2019-03-09 | 28           |
| 8         | 42        | 2019-03-30 | 99           |
| 14        | 161       | 2019-03-28 | 50           |
| 77        | 148       | 2019-01-22 | 15           |
| 100       | 87        | 2019-01-13 | 42           |
| 86        | 94        | 2019-03-24 | 58           |
| 23        | 26        | 2019-02-04 | 94           |
| 80        | 57        | 2019-02-25 | 96           |
| 98        | 48        | 2019-02-18 | 48           |
| 87        | 115       | 2019-02-25 | 91           |
| 30        | 6         | 2019-02-07 | 62           |
| 52        | 148       | 2019-02-19 | 73           |
| 5         | 178       | 2019-03-04 | 93           |
| 69        | 120       | 2019-03-23 | 79           |
| 95        | 76        | 2019-03-26 | 50           |
| 22        | 121       | 2019-02-28 | 25           |
| 65        | 96        | 2019-01-31 | 86           |
| 93        | 196       | 2019-02-23 | 76           |
| 63        | 12        | 2019-02-25 | 89           |
| 28        | 96        | 2019-02-25 | 91           |
| 3         | 6         | 2019-02-17 | 10           |
| 25        | 179       | 2019-03-26 | 34           |
| 91        | 17        | 2019-03-30 | 0            |
| 1         | 12        | 2019-03-10 | 39           |
| 69        | 7         | 2019-03-12 | 33           |
| 10        | 153       | 2019-03-21 | 49           |
| 41        | 98        | 2019-02-06 | 70           |
| 6         | 88        | 2019-03-07 | 23           |
| 46        | 81        | 2019-03-10 | 64           |
| 17        | 21        | 2019-02-09 | 70           |
| 66        | 151       | 2019-03-07 | 25           |
| 27        | 38        | 2019-03-11 | 80           |
| 84        | 91        | 2019-03-19 | 83           |
| 18        | 159       | 2019-03-09 | 72           |
| 72        | 188       | 2019-02-17 | 5            |
| 24        | 134       | 2019-01-08 | 23           |
| 28        | 190       | 2019-01-04 | 68           |
| 71        | 166       | 2019-03-20 | 13           |
| 59        | 156       | 2019-03-03 | 26           |
| 88        | 105       | 2019-02-21 | 5            |
| 42        | 55        | 2019-01-01 | 46           |
| 55        | 97        | 2019-03-08 | 76           |
| 1         | 3         | 2019-01-03 | 16           |
| 28        | 4         | 2019-02-22 | 50           |
| 63        | 148       | 2019-01-01 | 66           |
| 62        | 18        | 2019-01-10 | 56           |
| 8         | 160       | 2019-02-07 | 52           |
| 13        | 17        | 2019-01-19 | 49           |
| 27        | 91        | 2019-01-15 | 78           |
| 22        | 152       | 2019-03-14 | 33           |
| 73        | 56        | 2019-02-16 | 25           |
| 80        | 154       | 2019-01-31 | 29           |
| 4         | 12        | 2019-01-04 | 34           |
| 7         | 88        | 2019-02-08 | 8            |
| 66        | 42        | 2019-01-02 | 48           |
| 99        | 162       | 2019-01-01 | 88           |
| 19        | 117       | 2019-02-28 | 86           |
| 54        | 127       | 2019-01-01 | 11           |
| 67        | 102       | 2019-01-19 | 83           |
| 69        | 106       | 2019-01-30 | 16           |
| 85        | 129       | 2019-02-02 | 97           |
| 62        | 178       | 2019-03-06 | 39           |
| 87        | 38        | 2019-01-11 | 12           |
| 66        | 38        | 2019-03-04 | 46           |
| 93        | 175       | 2019-02-12 | 55           |
| 68        | 157       | 2019-01-03 | 98           |
| 38        | 157       | 2019-02-21 | 52           |
| 65        | 199       | 2019-03-26 | 72           |
| 61        | 167       | 2019-03-01 | 55           |
| 20        | 92        | 2019-02-15 | 30           |
| 61        | 45        | 2019-02-17 | 86           |
| 77        | 178       | 2019-01-26 | 52           |
| 34        | 61        | 2019-03-16 | 54           |
| 7         | 185       | 2019-02-21 | 86           |
| 8         | 91        | 2019-03-03 | 10           |
| 32        | 152       | 2019-03-03 | 46           |
| 83        | 5         | 2019-02-10 | 54           |
| 49        | 132       | 2019-01-10 | 57           |
| 32        | 56        | 2019-03-23 | 13           |
| 48        | 85        | 2019-03-15 | 26           |
| 50        | 23        | 2019-01-11 | 49           |
| 63        | 33        | 2019-03-08 | 29           |
| 14        | 115       | 2019-02-12 | 85           |
| 7         | 71        | 2019-01-04 | 45           |
| 57        | 126       | 2019-02-17 | 97           |
| 3         | 172       | 2019-03-19 | 30           |
| 76        | 147       | 2019-03-26 | 19           |
| 82        | 97        | 2019-03-26 | 78           |
| 84        | 51        | 2019-02-06 | 91           |
| 66        | 39        | 2019-02-27 | 7            |
| 69        | 168       | 2019-02-16 | 42           |
| 29        | 196       | 2019-02-13 | 7            |
| 90        | 11        | 2019-02-07 | 11           |
| 66        | 146       | 2019-03-11 | 44           |
| 90        | 125       | 2019-01-03 | 71           |
| 25        | 52        | 2019-01-08 | 71           |
| 42        | 156       | 2019-02-28 | 97           |
| 19        | 7         | 2019-01-18 | 85           |
| 16        | 135       | 2019-03-08 | 16           |
| 83        | 146       | 2019-01-07 | 47           |
| 67        | 46        | 2019-03-14 | 73           |
| 100       | 51        | 2019-02-17 | 29           |
| 67        | 112       | 2019-01-11 | 9            |
| 25        | 35        | 2019-02-21 | 89           |
| 65        | 54        | 2019-03-30 | 55           |
| 31        | 164       | 2019-01-27 | 1            |
| 49        | 75        | 2019-03-22 | 96           |
| 60        | 52        | 2019-03-23 | 59           |
| 65        | 16        | 2019-03-12 | 70           |
| 40        | 20        | 2019-01-02 | 76           |
| 24        | 93        | 2019-01-20 | 42           |
| 59        | 168       | 2019-03-09 | 83           |
| 7         | 174       | 2019-01-18 | 15           |
| 85        | 163       | 2019-01-19 | 44           |
| 69        | 126       | 2019-01-06 | 69           |
| 72        | 159       | 2019-02-18 | 84           |
| 19        | 37        | 2019-03-10 | 3            |
| 8         | 24        | 2019-01-22 | 92           |
| 63        | 111       | 2019-03-09 | 0            |
| 69        | 86        | 2019-02-21 | 72           |
| 87        | 100       | 2019-03-03 | 31           |
| 29        | 12        | 2019-01-07 | 70           |
| 88        | 111       | 2019-03-04 | 84           |
| 100       | 180       | 2019-03-24 | 55           |
| 57        | 197       | 2019-02-14 | 60           |
| 57        | 14        | 2019-02-09 | 79           |
| 59        | 60        | 2019-02-10 | 49           |
| 40        | 179       | 2019-03-09 | 17           |
| 68        | 198       | 2019-03-24 | 67           |
| 98        | 23        | 2019-01-17 | 75           |
| 18        | 90        | 2019-03-16 | 9            |
| 57        | 19        | 2019-03-20 | 13           |
| 25        | 154       | 2019-03-03 | 57           |
| 34        | 20        | 2019-03-14 | 1            |
| 41        | 78        | 2019-03-03 | 78           |
| 27        | 80        | 2019-02-27 | 96           |
| 58        | 77        | 2019-02-22 | 18           |
| 89        | 104       | 2019-02-03 | 97           |
| 18        | 84        | 2019-01-30 | 3            |
| 33        | 71        | 2019-01-24 | 43           |
| 28        | 193       | 2019-03-30 | 95           |
| 83        | 118       | 2019-02-04 | 19           |
| 94        | 101       | 2019-01-01 | 80           |
| 35        | 128       | 2019-01-01 | 72           |
| 74        | 190       | 2019-02-12 | 74           |
| 56        | 125       | 2019-03-19 | 33           |
| 90        | 18        | 2019-01-22 | 29           |
| 69        | 146       | 2019-02-22 | 20           |
| 3         | 4         | 2019-02-10 | 29           |
| 76        | 70        | 2019-02-06 | 40           |
| 16        | 87        | 2019-01-01 | 32           |
| 22        | 91        | 2019-01-12 | 73           |
| 27        | 16        | 2019-03-15 | 3            |
| 27        | 167       | 2019-02-23 | 42           |
| 95        | 91        | 2019-03-01 | 82           |
| 8         | 19        | 2019-03-29 | 31           |
| 95        | 111       | 2019-01-07 | 63           |
| 49        | 140       | 2019-02-18 | 29           |
| 6         | 38        | 2019-01-19 | 48           |
| 4         | 26        | 2019-03-01 | 51           |
| 53        | 193       | 2019-01-19 | 31           |
| 91        | 103       | 2019-01-18 | 60           |
| 95        | 37        | 2019-03-18 | 41           |
| 43        | 157       | 2019-01-10 | 45           |
| 67        | 63        | 2019-01-30 | 74           |
| 10        | 99        | 2019-03-02 | 64           |
| 53        | 91        | 2019-03-27 | 31           |
| 30        | 28        | 2019-01-27 | 2            |
| 18        | 152       | 2019-03-26 | 87           |
| 86        | 1         | 2019-02-15 | 37           |
| 60        | 76        | 2019-03-25 | 35           |
| 85        | 7         | 2019-01-16 | 74           |
| 98        | 49        | 2019-01-29 | 61           |
| 13        | 67        | 2019-02-06 | 8            |
| 47        | 50        | 2019-01-17 | 54           |
| 72        | 148       | 2019-01-28 | 78           |
| 72        | 134       | 2019-01-03 | 92           |
| 40        | 168       | 2019-03-29 | 46           |
| 32        | 184       | 2019-01-13 | 10           |
| 56        | 129       | 2019-02-19 | 97           |
| 33        | 11        | 2019-02-20 | 18           |
| 79        | 95        | 2019-03-23 | 20           |
| 92        | 186       | 2019-02-19 | 9            |
| 77        | 63        | 2019-03-01 | 72           |
| 42        | 164       | 2019-03-19 | 82           |
| 2         | 74        | 2019-02-27 | 47           |
| 55        | 198       | 2019-03-09 | 7            |
| 49        | 167       | 2019-03-01 | 41           |
| 18        | 135       | 2019-01-20 | 76           |
| 30        | 86        | 2019-01-22 | 64           |
| 94        | 188       | 2019-01-25 | 63           |
| 73        | 11        | 2019-02-17 | 69           |
| 38        | 179       | 2019-03-15 | 83           |
| 70        | 147       | 2019-01-21 | 95           |
| 20        | 43        | 2019-01-06 | 0            |
| 24        | 35        | 2019-03-05 | 81           |
| 96        | 34        | 2019-03-11 | 19           |
| 95        | 198       | 2019-01-21 | 96           |
| 89        | 81        | 2019-01-31 | 7            |
| 100       | 46        | 2019-02-03 | 38           |
| 75        | 45        | 2019-02-14 | 18           |
| 30        | 170       | 2019-02-11 | 43           |
| 45        | 61        | 2019-02-12 | 49           |
| 97        | 181       | 2019-03-18 | 52           |
| 21        | 138       | 2019-02-19 | 76           |
| 14        | 45        | 2019-01-11 | 39           |
| 26        | 120       | 2019-03-08 | 8            |
| 50        | 48        | 2019-03-13 | 11           |
| 94        | 131       | 2019-02-13 | 28           |
| 93        | 115       | 2019-01-07 | 5            |
| 41        | 144       | 2019-02-26 | 19           |
| 18        | 146       | 2019-02-26 | 45           |
| 33        | 133       | 2019-03-10 | 21           |
| 47        | 200       | 2019-01-26 | 54           |
| 62        | 105       | 2019-03-23 | 78           |
| 91        | 56        | 2019-01-24 | 35           |
| 16        | 16        | 2019-01-24 | 79           |
| 46        | 114       | 2019-02-17 | 70           |
| 49        | 197       | 2019-03-27 | 72           |
| 18        | 77        | 2019-03-30 | 95           |
| 14        | 124       | 2019-02-10 | 53           |
| 58        | 160       | 2019-01-18 | 85           |
| 84        | 194       | 2019-01-30 | 32           |
| 93        | 118       | 2019-02-25 | 61           |
| 79        | 187       | 2019-03-06 | 61           |
| 37        | 84        | 2019-03-07 | 91           |
| 83        | 35        | 2019-01-28 | 74           |
| 66        | 121       | 2019-02-23 | 73           |
| 32        | 99        | 2019-01-28 | 33           |
| 39        | 131       | 2019-01-04 | 74           |
| 77        | 45        | 2019-01-07 | 89           |
| 36        | 74        | 2019-01-06 | 64           |
| 68        | 161       | 2019-02-03 | 46           |
| 13        | 8         | 2019-02-17 | 72           |
| 33        | 183       | 2019-01-20 | 24           |
| 43        | 181       | 2019-02-03 | 83           |
| 78        | 100       | 2019-01-04 | 26           |
| 21        | 158       | 2019-01-27 | 14           |
| 6         | 120       | 2019-01-27 | 95           |
| 50        | 27        | 2019-01-29 | 1            |
| 15        | 162       | 2019-01-07 | 71           |
| 41        | 24        | 2019-01-15 | 80           |
| 64        | 195       | 2019-02-19 | 73           |
| 69        | 135       | 2019-01-20 | 18           |
| 77        | 105       | 2019-01-13 | 42           |
| 71        | 125       | 2019-03-16 | 42           |
| 34        | 134       | 2019-03-28 | 87           |
| 37        | 57        | 2019-01-17 | 45           |
| 29        | 1         | 2019-02-14 | 93           |
| 45        | 187       | 2019-03-13 | 59           |
| 70        | 176       | 2019-01-01 | 38           |
| 52        | 90        | 2019-02-14 | 30           |
| 55        | 139       | 2019-01-11 | 75           |
| 8         | 30        | 2019-03-23 | 37           |
| 54        | 140       | 2019-02-17 | 45           |
| 10        | 55        | 2019-02-12 | 75           |
| 11        | 46        | 2019-03-25 | 60           |
| 13        | 81        | 2019-01-21 | 70           |
| 39        | 174       | 2019-01-08 | 25           |
| 34        | 148       | 2019-02-02 | 48           |
| 71        | 128       | 2019-01-28 | 100          |
| 19        | 120       | 2019-03-09 | 54           |
| 1         | 41        | 2019-01-06 | 83           |
| 48        | 144       | 2019-01-31 | 23           |
| 18        | 49        | 2019-01-11 | 54           |
| 56        | 49        | 2019-02-20 | 61           |
| 35        | 78        | 2019-02-07 | 78           |
| 38        | 30        | 2019-02-14 | 57           |
| 27        | 40        | 2019-02-03 | 50           |
| 5         | 156       | 2019-03-02 | 76           |
| 30        | 188       | 2019-01-04 | 86           |
| 87        | 131       | 2019-02-15 | 20           |
| 22        | 182       | 2019-02-14 | 90           |
| 19        | 15        | 2019-03-30 | 26           |
| 98        | 147       | 2019-02-13 | 89           |
| 78        | 84        | 2019-02-16 | 79           |
| 79        | 88        | 2019-01-11 | 69           |
| 21        | 41        | 2019-01-15 | 39           |
| 57        | 51        | 2019-03-11 | 97           |
| 28        | 10        | 2019-03-09 | 58           |
| 32        | 23        | 2019-02-21 | 91           |
| 94        | 170       | 2019-03-22 | 68           |
| 23        | 25        | 2019-01-07 | 7            |
| 8         | 58        | 2019-02-28 | 92           |
| 44        | 180       | 2019-02-18 | 35           |
| 53        | 53        | 2019-03-30 | 15           |
| 68        | 107       | 2019-03-02 | 82           |
| 97        | 123       | 2019-03-08 | 53           |
| 40        | 38        | 2019-01-05 | 60           |
| 45        | 106       | 2019-01-04 | 13           |
| 40        | 29        | 2019-03-04 | 20           |
| 31        | 130       | 2019-02-07 | 1            |
| 100       | 87        | 2019-02-12 | 24           |
| 80        | 105       | 2019-03-05 | 62           |
| 47        | 138       | 2019-03-09 | 30           |
| 4         | 28        | 2019-03-17 | 27           |
| 88        | 42        | 2019-02-17 | 61           |
| 32        | 15        | 2019-01-23 | 21           |
| 88        | 18        | 2019-01-01 | 63           |
| 8         | 98        | 2019-03-18 | 16           |
| 24        | 155       | 2019-02-27 | 65           |
| 46        | 6         | 2019-02-08 | 8            |
| 69        | 77        | 2019-01-25 | 81           |
| 34        | 18        | 2019-02-27 | 11           |
| 68        | 73        | 2019-01-09 | 88           |
| 76        | 13        | 2019-01-21 | 73           |
| 73        | 144       | 2019-03-26 | 7            |
| 84        | 101       | 2019-01-12 | 98           |
| 84        | 200       | 2019-03-23 | 44           |
| 89        | 27        | 2019-02-08 | 37           |
| 100       | 15        | 2019-02-11 | 71           |
| 99        | 122       | 2019-03-22 | 53           |
| 43        | 44        | 2019-02-12 | 50           |
| 58        | 94        | 2019-03-09 | 6            |
| 68        | 65        | 2019-01-18 | 100          |
| 7         | 34        | 2019-01-30 | 84           |
| 90        | 40        | 2019-02-06 | 59           |
| 31        | 54        | 2019-02-13 | 60           |
| 5         | 8         | 2019-02-08 | 74           |
| 91        | 36        | 2019-01-09 | 66           |
| 86        | 62        | 2019-01-10 | 19           |
| 42        | 150       | 2019-01-25 | 40           |
| 44        | 161       | 2019-03-12 | 77           |
| 8         | 179       | 2019-01-26 | 28           |
| 13        | 27        | 2019-02-22 | 98           |
| 87        | 120       | 2019-01-04 | 15           |
| 21        | 34        | 2019-02-24 | 20           |
| 53        | 109       | 2019-03-15 | 89           |
| 54        | 41        | 2019-03-22 | 90           |
| 8         | 175       | 2019-03-22 | 6            |
| 94        | 193       | 2019-01-30 | 32           |
| 40        | 81        | 2019-01-12 | 17           |
| 64        | 34        | 2019-02-01 | 100          |
| 68        | 3         | 2019-03-21 | 32           |
| 19        | 72        | 2019-01-28 | 65           |
| 94        | 5         | 2019-01-24 | 59           |
| 74        | 14        | 2019-03-03 | 9            |
| 69        | 18        | 2019-01-16 | 54           |
| 54        | 191       | 2019-03-03 | 11           |
| 20        | 14        | 2019-03-05 | 68           |
| 62        | 2         | 2019-03-09 | 80           |
| 70        | 25        | 2019-03-08 | 34           |
| 44        | 46        | 2019-03-29 | 97           |
| 28        | 67        | 2019-02-01 | 55           |
| 39        | 183       | 2019-02-24 | 91           |
| 92        | 53        | 2019-01-28 | 86           |
| 60        | 9         | 2019-01-13 | 56           |
| 90        | 140       | 2019-01-20 | 80           |
| 89        | 44        | 2019-03-13 | 92           |
| 30        | 137       | 2019-03-25 | 55           |
| 27        | 138       | 2019-01-03 | 62           |
| 84        | 18        | 2019-03-03 | 28           |
| 17        | 63        | 2019-03-05 | 2            |
| 54        | 187       | 2019-01-07 | 3            |
| 27        | 50        | 2019-03-12 | 75           |
| 69        | 122       | 2019-02-08 | 40           |
| 64        | 24        | 2019-03-28 | 30           |
| 25        | 194       | 2019-01-14 | 63           |
| 28        | 90        | 2019-01-29 | 4            |
| 86        | 49        | 2019-03-07 | 66           |
| 76        | 197       | 2019-03-02 | 18           |
| 64        | 48        | 2019-01-02 | 74           |
| 12        | 174       | 2019-01-14 | 32           |
| 48        | 154       | 2019-01-08 | 76           |
| 72        | 15        | 2019-01-01 | 29           |
| 59        | 62        | 2019-03-11 | 89           |
| 8         | 179       | 2019-01-23 | 12           |
| 9         | 86        | 2019-02-24 | 48           |
| 88        | 191       | 2019-01-17 | 42           |
| 52        | 103       | 2019-01-25 | 37           |
| 2         | 36        | 2019-01-08 | 80           |
| 100       | 114       | 2019-03-03 | 16           |
| 51        | 108       | 2019-01-11 | 17           |
| 19        | 174       | 2019-02-25 | 44           |
| 5         | 36        | 2019-03-20 | 67           |
| 70        | 101       | 2019-02-05 | 28           |
| 73        | 2         | 2019-01-21 | 83           |
| 3         | 80        | 2019-03-20 | 12           |
| 80        | 123       | 2019-02-02 | 0            |
| 61        | 103       | 2019-02-01 | 81           |
| 23        | 156       | 2019-02-08 | 79           |
| 21        | 154       | 2019-03-29 | 58           |
| 95        | 36        | 2019-02-04 | 80           |
| 90        | 148       | 2019-02-26 | 97           |
| 96        | 49        | 2019-01-03 | 11           |
| 99        | 76        | 2019-01-27 | 78           |
| 1         | 80        | 2019-01-28 | 15           |
| 46        | 152       | 2019-01-27 | 87           |
| 71        | 118       | 2019-01-03 | 23           |
| 94        | 92        | 2019-03-04 | 16           |
| 60        | 12        | 2019-02-21 | 12           |
| 91        | 153       | 2019-02-26 | 15           |
| 96        | 5         | 2019-02-03 | 18           |
| 26        | 151       | 2019-03-14 | 95           |
| 51        | 197       | 2019-01-04 | 43           |
| 34        | 123       | 2019-01-08 | 0            |
| 77        | 152       | 2019-02-14 | 58           |
| 70        | 115       | 2019-01-30 | 98           |
| 9         | 122       | 2019-01-05 | 10           |
| 81        | 184       | 2019-02-11 | 83           |
| 72        | 38        | 2019-03-07 | 6            |
| 80        | 141       | 2019-03-07 | 76           |
| 55        | 64        | 2019-03-25 | 91           |
| 22        | 132       | 2019-02-09 | 50           |
| 64        | 53        | 2019-02-28 | 2            |
| 29        | 97        | 2019-02-03 | 73           |
| 17        | 57        | 2019-03-08 | 52           |
| 32        | 165       | 2019-01-26 | 26           |
| 82        | 50        | 2019-01-16 | 1            |
| 91        | 45        | 2019-03-11 | 68           |
| 40        | 99        | 2019-01-13 | 98           |
| 90        | 96        | 2019-01-24 | 60           |
| 55        | 21        | 2019-01-30 | 10           |
| 89        | 80        | 2019-01-18 | 43           |
| 18        | 57        | 2019-01-16 | 76           |
| 43        | 118       | 2019-03-02 | 86           |
| 4         | 49        | 2019-01-05 | 68           |
| 19        | 168       | 2019-01-21 | 11           |
| 82        | 150       | 2019-02-26 | 10           |
| 72        | 180       | 2019-01-24 | 66           |
| 63        | 194       | 2019-02-22 | 37           |
| 24        | 43        | 2019-02-13 | 11           |
| 49        | 38        | 2019-02-06 | 83           |
| 93        | 114       | 2019-02-05 | 55           |
| 80        | 178       | 2019-01-11 | 72           |
| 31        | 53        | 2019-02-02 | 49           |
| 48        | 71        | 2019-01-17 | 9            |
| 21        | 157       | 2019-01-10 | 1            |
| 48        | 176       | 2019-01-09 | 16           |
| 5         | 8         | 2019-01-12 | 18           |
| 86        | 14        | 2019-02-20 | 73           |
| 25        | 56        | 2019-01-12 | 93           |
| 71        | 12        | 2019-02-27 | 16           |
| 64        | 71        | 2019-03-23 | 72           |
| 95        | 65        | 2019-03-24 | 57           |
| 56        | 45        | 2019-02-03 | 73           |
| 55        | 56        | 2019-01-08 | 5            |
| 79        | 195       | 2019-01-31 | 91           |
| 27        | 4         | 2019-02-17 | 38           |
| 23        | 170       | 2019-02-25 | 0            |
| 75        | 90        | 2019-03-24 | 99           |
| 4         | 167       | 2019-03-02 | 90           |
| 60        | 180       | 2019-02-18 | 38           |
| 23        | 35        | 2019-03-16 | 70           |
| 68        | 39        | 2019-03-05 | 42           |
| 98        | 132       | 2019-02-21 | 16           |
| 21        | 185       | 2019-02-16 | 81           |
| 71        | 123       | 2019-02-23 | 61           |
| 14        | 76        | 2019-01-14 | 43           |
| 53        | 18        | 2019-01-16 | 61           |
| 26        | 198       | 2019-01-19 | 78           |
| 96        | 133       | 2019-01-08 | 76           |
| 7         | 19        | 2019-02-20 | 88           |
| 21        | 183       | 2019-03-28 | 93           |
| 70        | 96        | 2019-01-17 | 2            |
| 81        | 46        | 2019-03-11 | 51           |
| 58        | 133       | 2019-02-27 | 72           |
| 69        | 27        | 2019-01-27 | 40           |
| 38        | 149       | 2019-03-12 | 20           |
| 3         | 69        | 2019-01-21 | 32           |
| 73        | 144       | 2019-01-29 | 18           |
| 48        | 8         | 2019-01-30 | 28           |
| 61        | 193       | 2019-03-16 | 81           |
| 64        | 54        | 2019-02-05 | 84           |
| 12        | 193       | 2019-02-11 | 35           |
| 29        | 113       | 2019-02-24 | 40           |
| 97        | 173       | 2019-03-19 | 72           |
| 41        | 86        | 2019-03-06 | 12           |
| 77        | 186       | 2019-03-18 | 50           |
| 10        | 145       | 2019-03-27 | 89           |
| 71        | 120       | 2019-03-03 | 30           |
| 6         | 117       | 2019-03-16 | 60           |
| 6         | 65        | 2019-03-30 | 7            |
| 33        | 172       | 2019-01-23 | 22           |
| 67        | 168       | 2019-01-07 | 8            |
| 86        | 96        | 2019-01-31 | 38           |
| 42        | 48        | 2019-02-07 | 51           |
| 37        | 104       | 2019-03-24 | 68           |
| 70        | 90        | 2019-01-25 | 75           |
| 37        | 141       | 2019-03-26 | 92           |
| 87        | 146       | 2019-01-22 | 0            |
| 73        | 193       | 2019-01-07 | 74           |
| 58        | 195       | 2019-02-15 | 0            |
| 38        | 122       | 2019-01-27 | 17           |
| 77        | 139       | 2019-02-07 | 5            |
| 80        | 33        | 2019-01-20 | 76           |
| 92        | 63        | 2019-03-30 | 62           |
| 79        | 148       | 2019-03-30 | 79           |
| 25        | 114       | 2019-03-30 | 3            |
| 20        | 168       | 2019-01-27 | 51           |
| 54        | 64        | 2019-03-10 | 41           |
| 76        | 47        | 2019-03-30 | 8            |
| 34        | 179       | 2019-02-10 | 71           |
| 2         | 172       | 2019-02-17 | 92           |
| 66        | 116       | 2019-02-22 | 82           |
| 40        | 163       | 2019-02-15 | 54           |
| 1         | 50        | 2019-01-27 | 6            |
| 79        | 80        | 2019-01-26 | 49           |
| 22        | 72        | 2019-02-19 | 74           |
| 97        | 120       | 2019-03-17 | 76           |
| 55        | 151       | 2019-01-05 | 76           |
| 91        | 5         | 2019-02-11 | 91           |
| 68        | 148       | 2019-01-05 | 26           |
| 23        | 28        | 2019-01-28 | 6            |
| 20        | 101       | 2019-02-09 | 81           |
| 95        | 144       | 2019-01-27 | 10           |
| 42        | 168       | 2019-03-02 | 63           |
| 67        | 4         | 2019-01-08 | 32           |
| 63        | 26        | 2019-03-18 | 79           |
| 8         | 109       | 2019-01-28 | 41           |
| 31        | 79        | 2019-01-13 | 99           |
| 13        | 4         | 2019-02-20 | 17           |
| 92        | 168       | 2019-03-04 | 57           |
| 85        | 32        | 2019-03-30 | 92           |
| 65        | 5         | 2019-02-08 | 5            |
| 68        | 84        | 2019-03-19 | 49           |
| 95        | 86        | 2019-01-11 | 89           |
| 13        | 121       | 2019-01-20 | 75           |
| 29        | 49        | 2019-03-22 | 12           |
| 3         | 45        | 2019-02-02 | 24           |
| 86        | 135       | 2019-01-11 | 48           |
| 25        | 198       | 2019-02-20 | 34           |
| 70        | 7         | 2019-01-15 | 16           |
| 1         | 5         | 2019-03-14 | 68           |
| 60        | 196       | 2019-03-02 | 20           |
| 6         | 141       | 2019-01-26 | 66           |
| 55        | 148       | 2019-01-10 | 77           |
| 1         | 120       | 2019-01-01 | 40           |
| 96        | 38        | 2019-03-19 | 42           |
| 97        | 155       | 2019-01-25 | 40           |
| 6         | 167       | 2019-02-04 | 97           |
| 28        | 192       | 2019-02-08 | 31           |
| 17        | 184       | 2019-01-22 | 44           |
| 92        | 147       | 2019-03-14 | 32           |
| 71        | 193       | 2019-01-01 | 19           |
| 74        | 25        | 2019-02-16 | 66           |
| 52        | 38        | 2019-03-01 | 31           |
| 81        | 97        | 2019-01-23 | 12           |
| 28        | 118       | 2019-02-04 | 41           |
| 20        | 96        | 2019-02-03 | 20           |
| 45        | 16        | 2019-03-01 | 47           |
| 85        | 9         | 2019-03-11 | 64           |
| 3         | 75        | 2019-02-13 | 60           |
| 62        | 126       | 2019-02-24 | 68           |
| 93        | 67        | 2019-03-30 | 13           |
| 77        | 18        | 2019-03-20 | 5            |
| 65        | 187       | 2019-02-05 | 76           |
| 53        | 91        | 2019-03-16 | 8            |
| 65        | 65        | 2019-01-16 | 96           |
| 68        | 114       | 2019-03-23 | 31           |
| 36        | 46        | 2019-02-22 | 18           |
| 32        | 89        | 2019-02-19 | 94           |
| 14        | 165       | 2019-02-07 | 55           |
| 15        | 17        | 2019-01-26 | 19           |
| 12        | 125       | 2019-02-13 | 10           |
| 19        | 82        | 2019-01-17 | 67           |
| 57        | 10        | 2019-01-17 | 21           |
| 8         | 125       | 2019-01-15 | 73           |
| 1         | 164       | 2019-02-16 | 18           |
| 57        | 172       | 2019-03-17 | 49           |
| 75        | 1         | 2019-03-03 | 99           |
| 35        | 185       | 2019-03-15 | 41           |
| 94        | 119       | 2019-03-03 | 48           |
| 13        | 4         | 2019-03-14 | 13           |
| 64        | 71        | 2019-03-08 | 8            |
| 63        | 76        | 2019-02-08 | 44           |
| 42        | 153       | 2019-03-12 | 71           |
| 6         | 112       | 2019-01-14 | 1            |
| 54        | 161       | 2019-03-26 | 46           |
| 53        | 48        | 2019-01-21 | 77           |
| 98        | 35        | 2019-01-06 | 50           |
| 10        | 99        | 2019-01-20 | 38           |
| 22        | 32        | 2019-01-08 | 34           |
| 36        | 174       | 2019-03-27 | 1            |
| 24        | 65        | 2019-03-26 | 40           |
| 18        | 144       | 2019-03-11 | 50           |
| 4         | 34        | 2019-03-24 | 56           |
| 64        | 163       | 2019-03-25 | 86           |
| 68        | 193       | 2019-02-24 | 60           |
| 45        | 74        | 2019-02-09 | 4            |
| 39        | 143       | 2019-03-17 | 48           |
| 51        | 199       | 2019-02-12 | 60           |
| 34        | 66        | 2019-02-07 | 1            |
| 86        | 188       | 2019-03-16 | 54           |
| 82        | 120       | 2019-03-20 | 81           |
| 71        | 3         | 2019-03-02 | 89           |
| 10        | 80        | 2019-03-18 | 26           |
| 25        | 34        | 2019-02-07 | 74           |
| 45        | 130       | 2019-02-23 | 17           |
| 92        | 136       | 2019-01-10 | 95           |
| 30        | 165       | 2019-03-18 | 14           |
| 93        | 11        | 2019-01-02 | 32           |
| 55        | 45        | 2019-02-17 | 77           |
| 23        | 185       | 2019-01-23 | 30           |
| 17        | 80        | 2019-01-12 | 29           |
| 16        | 3         | 2019-02-13 | 3            |
| 69        | 68        | 2019-03-11 | 67           |
| 37        | 16        | 2019-03-25 | 61           |
| 98        | 140       | 2019-01-27 | 87           |
| 90        | 186       | 2019-02-02 | 31           |
| 29        | 41        | 2019-02-28 | 28           |
| 93        | 125       | 2019-02-01 | 96           |
| 87        | 136       | 2019-02-01 | 61           |
| 60        | 194       | 2019-02-04 | 68           |
| 21        | 194       | 2019-03-20 | 68           |
| 94        | 181       | 2019-02-07 | 15           |
| 19        | 108       | 2019-03-20 | 36           |
| 35        | 139       | 2019-03-28 | 60           |
| 87        | 171       | 2019-03-18 | 75           |
| 66        | 164       | 2019-02-21 | 10           |
| 52        | 107       | 2019-02-08 | 79           |
| 79        | 101       | 2019-03-04 | 5            |
| 5         | 72        | 2019-03-03 | 92           |
| 96        | 7         | 2019-02-26 | 70           |
| 23        | 112       | 2019-01-27 | 60           |
| 33        | 37        | 2019-03-17 | 69           |
| 81        | 56        | 2019-03-13 | 0            |
| 91        | 171       | 2019-01-19 | 1            |
| 91        | 64        | 2019-02-12 | 58           |
| 95        | 194       | 2019-01-02 | 86           |
| 17        | 188       | 2019-02-03 | 36           |
| 48        | 57        | 2019-02-26 | 25           |
| 93        | 165       | 2019-01-16 | 8            |
| 92        | 3         | 2019-01-16 | 5            |
| 27        | 56        | 2019-03-06 | 4            |
| 36        | 10        | 2019-03-29 | 31           |
| 88        | 83        | 2019-01-10 | 41           |
| 32        | 78        | 2019-02-02 | 84           |
| 2         | 8         | 2019-02-19 | 9            |
| 50        | 67        | 2019-03-29 | 72           |
| 84        | 7         | 2019-02-10 | 62           |
| 96        | 78        | 2019-02-27 | 93           |
| 12        | 171       | 2019-02-07 | 7            |
| 57        | 156       | 2019-02-19 | 37           |
| 11        | 113       | 2019-01-13 | 90           |
| 7         | 10        | 2019-03-26 | 30           |
| 1         | 15        | 2019-01-13 | 68           |
| 43        | 156       | 2019-03-13 | 27           |
| 71        | 103       | 2019-01-05 | 42           |
| 89        | 37        | 2019-02-01 | 84           |
| 73        | 23        | 2019-03-23 | 93           |
| 54        | 57        | 2019-01-19 | 13           |
| 11        | 13        | 2019-03-05 | 6            |
| 81        | 62        | 2019-01-16 | 62           |
| 86        | 20        | 2019-02-23 | 32           |
| 99        | 43        | 2019-01-25 | 39           |
| 82        | 176       | 2019-03-25 | 24           |
| 40        | 17        | 2019-02-02 | 57           |
| 35        | 153       | 2019-03-24 | 14           |
| 38        | 174       | 2019-02-07 | 13           |
| 91        | 95        | 2019-02-20 | 15           |
| 99        | 187       | 2019-01-18 | 99           |
| 28        | 93        | 2019-01-03 | 25           |
| 83        | 46        | 2019-01-12 | 50           |
| 78        | 86        | 2019-02-18 | 80           |
| 83        | 27        | 2019-01-02 | 88           |
| 59        | 80        | 2019-01-18 | 16           |
| 66        | 114       | 2019-02-04 | 31           |
| 53        | 79        | 2019-02-12 | 9            |
| 44        | 27        | 2019-01-27 | 44           |
| 25        | 105       | 2019-01-20 | 0            |
| 20        | 159       | 2019-01-23 | 40           |
| 89        | 1         | 2019-02-23 | 30           |
| 37        | 104       | 2019-03-06 | 92           |
| 97        | 145       | 2019-03-25 | 57           |
| 42        | 117       | 2019-02-16 | 63           |
| 33        | 180       | 2019-02-04 | 9            |
| 16        | 106       | 2019-02-18 | 73           |
| 89        | 117       | 2019-02-27 | 32           |
| 28        | 101       | 2019-01-05 | 88           |
| 98        | 120       | 2019-02-27 | 67           |
| 73        | 182       | 2019-01-03 | 17           |
| 32        | 98        | 2019-01-18 | 90           |
| 49        | 87        | 2019-02-22 | 8            |
| 93        | 95        | 2019-01-28 | 22           |
| 70        | 123       | 2019-03-27 | 5            |
| 68        | 4         | 2019-01-10 | 58           |
| 38        | 64        | 2019-02-20 | 74           |
| 32        | 115       | 2019-03-26 | 58           |
| 65        | 78        | 2019-02-26 | 74           |
| 70        | 131       | 2019-03-17 | 7            |
| 22        | 112       | 2019-01-28 | 91           |
| 38        | 144       | 2019-02-25 | 0            |
| 9         | 127       | 2019-01-04 | 55           |
| 80        | 2         | 2019-02-28 | 1            |
| 2         | 85        | 2019-03-01 | 90           |
| 66        | 182       | 2019-03-26 | 74           |
| 41        | 190       | 2019-02-04 | 45           |
| 35        | 83        | 2019-03-25 | 60           |
| 100       | 179       | 2019-03-10 | 32           |
| 65        | 83        | 2019-03-28 | 84           |
| 28        | 157       | 2019-02-06 | 64           |
| 26        | 105       | 2019-02-21 | 0            |
| 94        | 37        | 2019-01-15 | 81           |
| 74        | 25        | 2019-01-23 | 61           |
| 55        | 88        | 2019-01-07 | 77           |
| 2         | 80        | 2019-02-22 | 42           |
| 78        | 73        | 2019-03-01 | 62           |
| 99        | 78        | 2019-03-02 | 7            |
| 19        | 137       | 2019-03-25 | 73           |
| 4         | 97        | 2019-01-19 | 58           |
| 80        | 189       | 2019-03-22 | 77           |
| 80        | 195       | 2019-02-10 | 53           |
| 25        | 69        | 2019-03-18 | 52           |
| 58        | 124       | 2019-01-11 | 49           |
| 23        | 64        | 2019-03-26 | 55           |
| 80        | 79        | 2019-01-26 | 92           |
| 68        | 151       | 2019-01-20 | 98           |
| 7         | 4         | 2019-02-11 | 47           |
| 42        | 39        | 2019-03-08 | 84           |
| 6         | 12        | 2019-02-26 | 92           |
| 99        | 64        | 2019-01-13 | 76           |
| 22        | 54        | 2019-02-11 | 10           |
| 92        | 178       | 2019-01-23 | 97           |
| 59        | 86        | 2019-01-17 | 14           |
| 7         | 142       | 2019-01-26 | 8            |
| 45        | 146       | 2019-02-22 | 78           |
| 19        | 90        | 2019-03-26 | 59           |
| 8         | 118       | 2019-01-21 | 36           |
| 16        | 157       | 2019-01-20 | 33           |
| 40        | 67        | 2019-03-11 | 63           |
| 71        | 105       | 2019-02-03 | 75           |
| 29        | 189       | 2019-01-14 | 91           |
| 12        | 38        | 2019-01-12 | 27           |
| 20        | 67        | 2019-01-22 | 43           |
| 54        | 91        | 2019-03-25 | 63           |
| 75        | 142       | 2019-03-07 | 82           |
| 39        | 159       | 2019-03-11 | 18           |
| 8         | 11        | 2019-01-17 | 34           |
| 30        | 186       | 2019-02-10 | 20           |
| 59        | 195       | 2019-02-14 | 5            |
| 74        | 124       | 2019-01-16 | 13           |
| 78        | 8         | 2019-01-08 | 85           |
| 86        | 25        | 2019-03-04 | 37           |
| 47        | 106       | 2019-03-23 | 98           |
| 99        | 147       | 2019-02-21 | 36           |
| 72        | 72        | 2019-01-19 | 63           |
| 69        | 6         | 2019-02-13 | 41           |
| 64        | 164       | 2019-01-11 | 98           |
| 94        | 175       | 2019-03-21 | 88           |
| 17        | 134       | 2019-03-30 | 18           |
| 46        | 173       | 2019-01-18 | 73           |
| 69        | 142       | 2019-02-26 | 24           |
| 44        | 138       | 2019-02-24 | 31           |
| 49        | 150       | 2019-02-01 | 46           |
| 40        | 97        | 2019-03-24 | 85           |
| 98        | 73        | 2019-02-14 | 33           |
| 2         | 51        | 2019-03-20 | 24           |
| 45        | 101       | 2019-02-14 | 98           |
| 40        | 187       | 2019-01-22 | 85           |
| 25        | 176       | 2019-02-11 | 37           |
| 94        | 74        | 2019-02-01 | 57           |
| 96        | 129       | 2019-03-13 | 0            |
| 77        | 163       | 2019-02-20 | 100          |
| 43        | 9         | 2019-01-26 | 95           |
| 68        | 148       | 2019-02-13 | 4            |
| 26        | 59        | 2019-01-12 | 15           |
| 41        | 75        | 2019-02-11 | 99           |
| 30        | 102       | 2019-01-11 | 41           |
| 3         | 168       | 2019-01-15 | 27           |
| 34        | 38        | 2019-02-03 | 63           |
| 27        | 44        | 2019-01-31 | 23           |
| 26        | 174       | 2019-02-03 | 32           |
| 95        | 96        | 2019-02-03 | 44           |
| 83        | 76        | 2019-03-28 | 72           |
| 20        | 165       | 2019-01-02 | 64           |
| 80        | 68        | 2019-03-29 | 30           |
| 10        | 14        | 2019-02-23 | 10           |
| 40        | 24        | 2019-03-16 | 62           |
| 25        | 4         | 2019-02-03 | 48           |
| 9         | 1         | 2019-01-06 | 67           |
| 6         | 122       | 2019-02-20 | 92           |
| 50        | 39        | 2019-01-05 | 46           |
| 28        | 153       | 2019-01-26 | 59           |
| 8         | 121       | 2019-03-09 | 31           |
| 23        | 176       | 2019-01-25 | 21           |
| 17        | 133       | 2019-02-18 | 81           |
| 45        | 52        | 2019-03-11 | 24           |
| 74        | 181       | 2019-02-03 | 96           |
| 97        | 177       | 2019-02-25 | 0            |
| 4         | 59        | 2019-03-10 | 87           |
| 84        | 173       | 2019-03-24 | 42           |
| 48        | 154       | 2019-03-01 | 63           |
| 44        | 13        | 2019-01-25 | 42           |
| 46        | 84        | 2019-02-22 | 95           |
| 84        | 68        | 2019-02-03 | 74           |
| 9         | 99        | 2019-02-16 | 48           |
| 43        | 62        | 2019-03-22 | 61           |
| 85        | 48        | 2019-01-22 | 80           |
| 45        | 184       | 2019-03-15 | 1            |
| 42        | 7         | 2019-02-14 | 7            |
| 32        | 199       | 2019-02-13 | 96           |
| 55        | 184       | 2019-02-06 | 94           |
| 1         | 161       | 2019-02-21 | 8            |
| 96        | 20        | 2019-03-15 | 91           |
| 13        | 123       | 2019-02-01 | 90           |
| 84        | 150       | 2019-01-19 | 84           |
| 96        | 31        | 2019-02-25 | 28           |
| 91        | 152       | 2019-03-29 | 84           |
| 64        | 12        | 2019-01-25 | 8            |
| 71        | 94        | 2019-01-09 | 44           |
| 56        | 161       | 2019-03-04 | 69           |
| 36        | 122       | 2019-02-03 | 9            |
| 100       | 54        | 2019-03-22 | 75           |
| 23        | 117       | 2019-03-08 | 10           |
| 32        | 148       | 2019-01-17 | 4            |
| 34        | 45        | 2019-02-04 | 38           |
| 98        | 39        | 2019-01-05 | 43           |
| 32        | 199       | 2019-03-02 | 44           |
| 96        | 198       | 2019-03-05 | 48           |
| 6         | 186       | 2019-01-16 | 5            |
| 74        | 130       | 2019-03-04 | 75           |
| 57        | 62        | 2019-01-20 | 23           |
| 2         | 80        | 2019-01-04 | 90           |
| 77        | 84        | 2019-03-07 | 56           |
| 26        | 22        | 2019-03-01 | 14           |
| 13        | 80        | 2019-01-16 | 67           |
| 23        | 91        | 2019-02-10 | 21           |
| 13        | 60        | 2019-01-12 | 97           |
| 51        | 107       | 2019-01-19 | 1            |
| 82        | 34        | 2019-03-17 | 40           |
| 12        | 166       | 2019-01-11 | 46           |
| 2         | 43        | 2019-02-05 | 50           |
| 39        | 9         | 2019-03-09 | 97           |
| 19        | 165       | 2019-02-27 | 27           |
| 72        | 142       | 2019-01-16 | 85           |
| 99        | 38        | 2019-03-01 | 26           |
| 28        | 4         | 2019-03-29 | 55           |
| 60        | 118       | 2019-03-10 | 57           |
| 75        | 74        | 2019-03-16 | 78           |
| 7         | 90        | 2019-03-02 | 19           |
| 35        | 115       | 2019-03-07 | 73           |
| 56        | 159       | 2019-01-18 | 94           |
| 69        | 166       | 2019-01-19 | 71           |
| 44        | 10        | 2019-01-10 | 46           |
| 13        | 178       | 2019-03-15 | 71           |
| 53        | 130       | 2019-01-12 | 80           |
| 49        | 93        | 2019-03-23 | 79           |
| 56        | 180       | 2019-01-10 | 42           |
| 82        | 172       | 2019-01-05 | 94           |
| 71        | 57        | 2019-02-17 | 31           |
| 22        | 146       | 2019-02-12 | 15           |
| 10        | 122       | 2019-03-01 | 14           |
| 22        | 136       | 2019-03-04 | 4            |
| 74        | 31        | 2019-02-18 | 59           |
| 4         | 66        | 2019-02-09 | 61           |
| 62        | 128       | 2019-03-30 | 34           |
| 58        | 187       | 2019-02-16 | 81           |
| 10        | 130       | 2019-03-29 | 9            |
| 16        | 47        | 2019-01-17 | 86           |
| 87        | 10        | 2019-02-14 | 2            |
| 44        | 71        | 2019-01-23 | 81           |
| 13        | 26        | 2019-02-04 | 88           |
| 37        | 80        | 2019-02-09 | 28           |
| 63        | 72        | 2019-02-11 | 63           |
| 38        | 200       | 2019-03-25 | 29           |
| 21        | 91        | 2019-03-15 | 45           |
| 86        | 6         | 2019-03-01 | 22           |
| 34        | 74        | 2019-03-20 | 35           |
| 63        | 137       | 2019-03-05 | 26           |
| 48        | 173       | 2019-03-14 | 54           |
| 59        | 36        | 2019-02-13 | 75           |
| 65        | 139       | 2019-01-30 | 29           |
| 66        | 186       | 2019-02-16 | 73           |
| 90        | 46        | 2019-03-04 | 38           |
| 7         | 82        | 2019-03-21 | 27           |
| 40        | 63        | 2019-02-20 | 10           |
| 86        | 40        | 2019-03-19 | 51           |
| 77        | 168       | 2019-03-16 | 34           |
| 77        | 92        | 2019-02-09 | 31           |
| 10        | 144       | 2019-02-14 | 37           |
| 76        | 28        | 2019-01-30 | 85           |
| 38        | 98        | 2019-03-13 | 87           |
| 77        | 127       | 2019-02-26 | 89           |
| 38        | 196       | 2019-02-15 | 96           |
| 6         | 200       | 2019-01-24 | 0            |
| 90        | 105       | 2019-02-15 | 69           |
| 3         | 200       | 2019-01-13 | 55           |
| 94        | 18        | 2019-03-27 | 6            |
| 92        | 79        | 2019-03-13 | 74           |
| 6         | 191       | 2019-03-08 | 37           |
| 84        | 129       | 2019-03-12 | 18           |
| 15        | 199       | 2019-02-07 | 22           |
| 55        | 134       | 2019-03-11 | 53           |
| 54        | 92        | 2019-01-03 | 15           |
| 93        | 39        | 2019-02-26 | 26           |
| 79        | 40        | 2019-02-20 | 82           |
| 35        | 127       | 2019-03-14 | 63           |
| 94        | 28        | 2019-03-08 | 83           |
| 61        | 11        | 2019-02-07 | 1            |
| 66        | 48        | 2019-01-25 | 55           |
| 76        | 156       | 2019-02-02 | 67           |
| 41        | 106       | 2019-01-01 | 37           |
| 37        | 148       | 2019-03-21 | 7            |
| 78        | 129       | 2019-02-13 | 86           |
| 21        | 133       | 2019-03-04 | 64           |
| 99        | 56        | 2019-01-17 | 46           |
| 98        | 20        | 2019-01-26 | 31           |
| 65        | 29        | 2019-03-05 | 15           |
| 70        | 79        | 2019-03-30 | 97           |
| 76        | 1         | 2019-01-06 | 43           |
| 36        | 58        | 2019-03-02 | 76           |
| 99        | 19        | 2019-01-14 | 27           |
| 56        | 77        | 2019-01-19 | 36           |
| 12        | 7         | 2019-02-26 | 19           |
| 23        | 187       | 2019-02-18 | 93           |
| 11        | 167       | 2019-02-16 | 90           |
| 22        | 64        | 2019-02-27 | 23           |
| 21        | 52        | 2019-01-03 | 33           |
| 59        | 138       | 2019-01-21 | 65           |
| 72        | 164       | 2019-02-10 | 77           |
| 6         | 92        | 2019-03-21 | 61           |
| 98        | 8         | 2019-02-11 | 12           |
| 70        | 180       | 2019-03-12 | 2            |
| 12        | 5         | 2019-01-30 | 6            |
| 48        | 132       | 2019-02-15 | 88           |
| 47        | 110       | 2019-02-09 | 86           |
| 64        | 96        | 2019-02-03 | 51           |
| 1         | 133       | 2019-02-15 | 18           |
| 6         | 121       | 2019-03-01 | 62           |
| 70        | 151       | 2019-01-12 | 98           |
| 58        | 197       | 2019-02-10 | 96           |
| 8         | 133       | 2019-01-06 | 65           |
| 52        | 124       | 2019-02-23 | 91           |
| 83        | 118       | 2019-02-03 | 25           |
| 39        | 188       | 2019-01-03 | 36           |
| 26        | 28        | 2019-03-27 | 70           |
| 92        | 68        | 2019-03-03 | 53           |
| 51        | 190       | 2019-02-23 | 60           |
| 91        | 180       | 2019-02-21 | 27           |
| 34        | 181       | 2019-03-24 | 8            |
| 77        | 91        | 2019-01-14 | 41           |
| 5         | 59        | 2019-03-26 | 86           |
| 89        | 116       | 2019-03-27 | 63           |
| 9         | 94        | 2019-03-10 | 87           |
| 71        | 173       | 2019-01-26 | 43           |
| 3         | 131       | 2019-03-08 | 77           |
| 84        | 133       | 2019-03-11 | 3            |
| 46        | 83        | 2019-03-11 | 11           |
| 49        | 199       | 2019-01-26 | 34           |
| 30        | 76        | 2019-02-16 | 34           |
| 20        | 156       | 2019-02-06 | 89           |
| 23        | 108       | 2019-01-01 | 23           |
| 47        | 69        | 2019-02-13 | 93           |
| 60        | 63        | 2019-03-15 | 44           |
| 85        | 99        | 2019-02-27 | 45           |
| 78        | 34        | 2019-01-16 | 87           |
| 58        | 31        | 2019-02-25 | 52           |


"""

In [8]:
import pandas as pd
from io import StringIO

# Sample data in the form of a string


# Parse the data string into a pandas DataFrame
df = pd.read_csv(StringIO(data_str), sep='|', skiprows=1, skipinitialspace=True, skipfooter=1, engine='python', header=None)


# Drop any empty rows
df.dropna(how='all', inplace=True)
df = df.drop(columns=[0, 5])
df.columns = ["player_id","device_id","event_date","games_played"]
df = df.loc[2:]

activity = df.astype(
    {'player_id': 'Int64', 'device_id': 'Int64', 'event_date': 'datetime64[ns]', 'games_played': 'Int64'})


df_person = spark.createDataFrame(activity)
df_person.createOrReplaceTempView("Activity_1")


In [10]:
activity

,player_id,device_id,event_date,games_played
2,5,17,2019-01-17,44
3,31,43,2019-01-21,41
4,52,89,2019-03-09,44
5,14,154,2019-02-13,87
6,96,46,2019-03-22,77
...,...,...,...,...
997,47,69,2019-02-13,93
998,60,63,2019-03-15,44
999,85,99,2019-02-27,45
1000,78,34,2019-01-16,87


In [16]:
query_2 = """
    select
        ROUND(COUNT(DISTINCT player_id) / (SELECT COUNT(DISTINCT player_id) FROM Activity_1), 2) AS fraction
    from (select
        a.player_id,
        a.device_id,
        a.event_date,
        a.games_played,
        min(a.event_date) over (partition by player_id) as first_date_login
    from Activity_1 a) t
    where
        date_sub(event_date, 1) = first_date_login

"""
    # where 
    #     date_sub(event_date, 1) = first_date_login 

In [17]:
query_3 = """
    select count(distinct player_id) from Activity_1
"""

In [18]:
query_4 = """
    SELECT
        ROUND(COUNT(DISTINCT player_id) / (SELECT COUNT(DISTINCT player_id) FROM Activity_1), 2) AS fraction
    FROM
        Activity_1
    WHERE
        (player_id, DATE_SUB(event_date, 1))
        IN (
            SELECT player_id, MIN(event_date) AS first_login FROM Activity_1 GROUP BY player_id
        )
"""

In [19]:
spark.sql(query_2).show()

+--------+
|fraction|
+--------+
|    0.12|
+--------+

